In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(temperature=0.1,
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영화제목을 이모티콘으로 표현"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke(
        {
            "question": question
        }
    )
    memory.save_context({"input":question}, {"output":result.content})
    print(result)

invoke_chain("인셉션")

💭🌀🕰️content='💭🌀🕰️'


In [3]:
invoke_chain("인터스텔라")

🚀🌌🕰️content='🚀🌌🕰️'


In [4]:
invoke_chain("앞에 물어본 영화는?")

탑건content='탑건'


In [5]:
invoke_chain("앞에 물어본 영화들은?")

탑건, 대부, 인셉션, 인터스텔라The human mentions "Inception."content='탑건, 대부, 인셉션, 인터스텔라'


In [8]:
load_memory('history')

[SystemMessage(content='The human mentions "Inception."'),
 AIMessage(content='💭🌀🕰️'),
 HumanMessage(content='인터스텔라'),
 AIMessage(content='🚀🌌🕰️'),
 HumanMessage(content='앞에 물어본 영화는?'),
 AIMessage(content='탑건'),
 HumanMessage(content='앞에 물어본 영화들은?'),
 AIMessage(content='탑건, 대부, 인셉션, 인터스텔라')]